In [1]:
import torch

In [2]:
torch.cuda.is_available()

False

In [3]:
torch.cuda.get_device_name(0)

AssertionError: Torch not compiled with CUDA enabled

In [4]:
torch.cuda.mem_get_info()

AssertionError: Torch not compiled with CUDA enabled

In [5]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

if device.type=='cuda':
    print(torch.cuda.get_device_name(0))

cpu


In [6]:
CLASSES=10
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [7]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
#import matplotlib.pyplot as plt
import numpy as np

In [16]:
class Net1(nn.Module):
    def __init__(self):
        super().__init__()
        # Layer 1
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=96,kernel_size=11,stride=4)
        # Layer 2
        self.conv2 = nn.Conv2d(in_channels=96,out_channels=96,kernel_size=1)
        # Layer 3
        self.conv3 = self.conv2
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=2)
        # Layer 4
        self.conv4 = nn.Conv2d(in_channels=96,out_channels=256,kernel_size=11,stride=4,padding=2)
        # Layer 5
        self.conv5 = nn.Conv2d(in_channels=256,out_channels=256,kernel_size=1)
        # Layer 6
        self.conv6 = self.conv5
        self.pool2 = nn.MaxPool2d(kernel_size=3,stride=2)
        # Layer 7
        self.conv7 = nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3,stride=1,padding=1)
        # Layer 8
        self.conv8 = nn.Conv2d(in_channels=384,out_channels=384,kernel_size=1)
        # Layer 9
        self.conv9 = self.conv8
        # Layer 10
        self.conv10 = nn.Conv2d(in_channels=384,out_channels=10,kernel_size=3,stride=1,padding=1)
        # Layer 11
        self.conv11 = nn.Conv2d(in_channels=10,out_channels=10,kernel_size=1)
        # Layer 12
        self.conv12 = self.conv11
        self.pool3 = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        # Layer 1
        x = F.relu(self.conv1(x))
        # Layer 2
        x = F.relu(self.conv2(x))
        # Layer 3
        x = F.relu(self.conv3(x))
        x = self.pool1(x)
        x = F.dropout(x, 0.5)
        # Layer 4
        x = F.relu(self.conv4(x))
        # Layer 5
        x = F.relu(self.conv5(x))
        # Layer 6
        x = F.relu(self.conv6(x))
        x = self.pool2(x)
        x = F.dropout(x, 0.5)
        # Layer 7
        x = F.relu(self.conv7(x))
        # Layer 8
        x = F.relu(self.conv8(x))
        # Layer 9
        x = F.relu(self.conv9(x))
        x = F.dropout(x, 0.5)
        # Layer 10
        x = F.relu(self.conv10(x))
        # Layer 11
        x = F.relu(self.conv11(x))
        # Layer 12
        x = F.relu(self.conv12(x))
        x = self.pool3(x)
        # Flatten into tensor for use
        x = torch.flatten(x, start_dim=1)

        return x

In [17]:
transform_conf=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [18]:
BATCH_SIZE=16
train_dataset=datasets.MNIST('/users/gwitske/hw2/data/',train=True,download=True,transform=transform_conf,)
test_dataset=datasets.MNIST('/users/gwitske/hw2/data/',train=False,download=True,transform=transform_conf)

In [19]:
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=BATCH_SIZE,shuffle=True)

In [20]:
model=Net1().to(device)
optimizer=optim.Adam(params=model.parameters(),lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [21]:
def train(model,device,train_loader,optimizer,epochs):
    print("inside train")
    model.train()
    for batch_ids, (img, classes) in enumerate(train_loader):
        classes=classes.type(torch.LongTensor)
        img,classes=img.to(device),classes.to(device)
        torch.autograd.set_detect_anomaly(True)     
        optimizer.zero_grad()
        output=model(img)
        loss = loss_fn(output,classes)                
        
        loss.backward()
        optimizer.step()
    if(batch_ids +1) % 2 == 0:
        print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epochs, batch_ids* len(img), len(train_loader.dataset),
            100.*batch_ids / len(train_loader),loss.item()))

In [22]:
def test(model, device, test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for img,classes in test_loader:
            img,classes=img.to(device), classes.to(device)
            y_hat=model(img)
            test_loss+=F.nll_loss(y_hat,classes,reduction='sum').item()
            _,y_pred=torch.max(y_hat,1)
            correct+=(y_pred==classes).sum().item()
        test_loss/=len(test_dataset)
        print("\n Test set: Average loss: {:.0f},Accuracy:{}/{} ({:.0f}%)\n".format(
            test_loss,correct,len(test_dataset),100.*correct/len(test_dataset)))
        print('='*30)

In [23]:
if __name__=='__main__':
    seed=42
    EPOCHS=2
    
    for epoch in range(1,EPOCHS+1):
        train(model,device,train_loader,optimizer,epoch)
        test(model,device,test_loader)

inside train
Train Epoch: 1 [59984/60000 (100%)]	Loss: 0.846296

 Test set: Average loss: -12,Accuracy:9482/10000 (95%)

inside train
Train Epoch: 2 [59984/60000 (100%)]	Loss: 0.044791

 Test set: Average loss: -14,Accuracy:9697/10000 (97%)



In [25]:
class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        # Output size = (Input size - Kernel size + 2 * Padding) / Stride + 1
        # MNIST Image Size: 224 x 224 x 1

        # Layer 1
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=96,kernel_size=11,stride=4) # Output size = (224 - 11 + 2 * 0) / 4 + 1 = 54
        # Layer 2
        self.conv2 = nn.Conv2d(in_channels=96,out_channels=96,kernel_size=1) # Output size = (54 - 1 + 2 * 0) / 1 + 1 = 54
        # Layer 3
        self.conv3 = self.conv2 # Output size = (54 - 1 + 2 * 0) / 1 + 1 = 54
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=2) # Output size = (54 - 3 + 2 * 0) / 2 + 1 = 26
        # Layer 4
        self.conv4 = nn.Conv2d(in_channels=96,out_channels=256,kernel_size=11,stride=4,padding=2) # Output size = (26 - 11 + 2 * 2) / 4 + 1 = 5
        # Layer 5
        self.conv5 = nn.Conv2d(in_channels=256,out_channels=256,kernel_size=1) # Output size = (5 - 1 + 2 * 0) / 1 + 1 = 5
        # Layer 6
        self.conv6 = self.conv5 # Output size = (5 - 1 + 2 * 0) / 1 + 1 = 5
        self.pool2 = nn.MaxPool2d(kernel_size=3,stride=2) # Output size = (5 - 3 + 2 * 0) / 2 + 1 = 2
        # Layer 7
        self.conv7 = nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3,stride=1,padding=1) # Output size = (2 - 3 + 2 * 1) / 1 + 1 = 2
        # Layer 8
        self.conv8 = nn.Conv2d(in_channels=384,out_channels=384,kernel_size=1) # Output size = (2 - 1 + 2 * 0) / 1 + 1 = 2
        # Layer 9
        self.conv9 = self.conv8 # Output size = (2 - 1 + 2 * 0) / 1 + 1 = 2
        # Layer 10
        self.fc1 = nn.Linear(in_features=(2*2*384),out_features=4096)
        # Layer 11
        self.fc2 = nn.Linear(in_features=(4096),out_features=4096)
        # Layer 12 (not really layer)
        self.fc3 = nn.Linear(in_features=(4096),out_features=CLASSES)

    def forward(self, x):
        # Layer 1
        x = F.relu(self.conv1(x))
        # Layer 2
        x = F.relu(self.conv2(x))
        # Layer 3
        x = F.relu(self.conv3(x))
        x = self.pool1(x)
        x = F.dropout(x, 0.5)
        # Layer 4
        x = F.relu(self.conv4(x))
        # Layer 5
        x = F.relu(self.conv5(x))
        # Layer 6
        x = F.relu(self.conv6(x))
        x = self.pool2(x)
        x = F.dropout(x, 0.5)
        # Layer 7
        x = F.relu(self.conv7(x))
        # Layer 8
        x = F.relu(self.conv8(x))
        # Layer 9
        x = F.relu(self.conv9(x))
        x = F.dropout(x, 0.5)
        # Flatten into tensor for use in fc layers
        x = torch.flatten(x, start_dim=1)
        # Layer 10
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5)
        # Layer 11
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.5)
        # Layer 12 (Not really layer)
        x = self.fc3(x)

        return x

In [26]:
model=Net2().to(device)
optimizer=optim.Adam(params=model.parameters(),lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [27]:
if __name__=='__main__':
    seed=42
    EPOCHS=2
    
    for epoch in range(1,EPOCHS+1):
        train(model,device,train_loader,optimizer,epoch)
        test(model,device,test_loader)

inside train
Train Epoch: 1 [59984/60000 (100%)]	Loss: 0.588614

 Test set: Average loss: -7,Accuracy:9716/10000 (97%)

inside train
Train Epoch: 2 [59984/60000 (100%)]	Loss: 0.045152

 Test set: Average loss: -8,Accuracy:9749/10000 (97%)

